# Automate Event Planning 🎉

## Introduction 
In this notebook, an insight will be given about `how to automate event planning tasks using crewAI tools and agents`. The goal is to streamline the process of organizing an event by leveraging AI to handle tasks such as finding a venue, managing logistics, and marketing the event.

> _Note_: If you're running this notebook on your own machine, you can install the following:
```Py
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Suppress warnings for a cleaner output
import warnings
warnings.filterwarnings('ignore')

### Import libraries, APIs and LLM

In [ ]:
# Importing necessary classes from the crewai library
from crewai import Agent, Crew, Task

> _Note_: _Agent_ represents an agent with specific roles and goals. _Task_ represents a task that the agent needs to perform. _Crew_ represents a group of agents working together on tasks.    

In [ ]:
# Import the required libraries
import os
from utils import get_openai_api_key,get_serper_api_key

# Fetch the OpenAI API key using a utility function and store it in a variable
openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

# Fetch the Serper API key using a utility function and set it as an environment variable
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
# Import crewAI tools for web scraping and search functionality
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

**Note** _ScrapeWebsiteTool_ is a Tool for scraping data from websites. The _SerperDevTool_ is a  Tool for performing searches using the Serper API

## Creating Agents

- Agent 1: Venue Coordinator

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

- Agent 2: Logistics Manager

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

- Agent 3: Marketing and Communications Agent

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel

# Define a Pydantic model for venue details 
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

- Venue Task

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- Logistics Task

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

**Note** By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

- Marketing Task

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: To find the best venue for the Tech Innovation Conference in San Francisco, I need to search for venues that can accommodate such events and then check each of their availability, facilities, and cost to ensure they meet our requirements.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in San Francisco"} 


Search results: Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snippet: The World Agri-Tech Innovation Summit is the stand out event with curated leaders, innovators and investors from across the AgTech landscape.
---
Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: To effectively market the Tech Innovation Conference and engage potential attendees, it's crucial to understand the venue's amenities and any promotional materials or past events hosted there. This will help in tailoring the marketing message and possibly leveraging the venue's reputation.

Action: Read website content
Action Input: {"website_url": "https://www.marriott.com/hotels/travel/sfodt-san-francisco-marriott-marquis/"} 

Access Denied
Access Denied
You don't have permission to access "http://www.marriott.com/en-us/hotels/sfodt-san-francisco-marriott-marquis/overview/" on this server.
Reference #18.9f3e1602.1739883071.e69ff11
https://errors.edgesuite.net/18.9f3e1602.1739883071.e69ff11

Thought: Since I couldn't access the Marriott Marquis website directly, I should search the internet to gather information about the venue's amenities, services, and any notable events that have occurr

{'address': '780 Mission Street, San Francisco, CA 94103',
 'booking_status': 'Contact directly for availability and booking',
 'capacity': 5500,
 'name': 'San Francisco Marriott Marquis'}


- Display the generated `marketing_report.md` file.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md

**Explanation output** The report (see marketing_report.md) outlines the marketing strategy for the upcoming Tech Innovation Conference to be held at the San Francisco Marriott Marquis. The goal is to attract IT professionals interested in emerging technologies like artificial intelligence through targeted campaigns and strategic partnerships.

## Conclusion
By automating the event planning process with crewAI tools and agents, you can efficiently manage key tasks such as venue selection, logistics coordination, and marketing. This approach `not only streamlines operations but also ensures a well-organized and successful event`.